# EDA Industry

Workflow: 0 

Goal: To create plots for Exploration page.

In [1]:
import os, sys, time, json
import pandas as PD
import numpy as NP

import plotly.graph_objects as GO
import plotly.figure_factory as FF
import plotly.subplots as SB

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last'

In [3]:
os.chdir(os.getenv('PWD'))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

In [4]:
import movies.models.models as MD
import movies.models.exploration as XT
import movies.models.analysis as NL

## Score vs Profit

In [5]:
XT.Explore.GetScoreProfitPlot()

In [6]:
master_ls = list(MD.MasterMovie.objects.values())
master_df = PD.DataFrame(master_ls)
master_df = master_df.drop(columns=['id', 'OriginalTitle', 'Poster', 'Synopsis', 'Indeces'])
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17668 entries, 0 to 17667
Data columns (total 15 columns):
Movie_ID     17668 non-null int64
Title        17668 non-null object
Year         17668 non-null object
Rating       8454 non-null object
Companies    16431 non-null object
Country      17627 non-null object
Language     17667 non-null object
RunTime      17664 non-null float64
Crew         16375 non-null object
Cast         14064 non-null object
Genres       17554 non-null object
Budget       4782 non-null float64
Gross        5530 non-null float64
ScoreImdb    17211 non-null float64
VotesImdb    17211 non-null float64
dtypes: float64(5), int64(1), object(9)
memory usage: 2.0+ MB


In [7]:
# drop nan's 

scoreprofit_df = master_df.dropna(axis=0, subset=['Budget', 'Gross', 'ScoreImdb'])
scoreprofit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2321 entries, 1 to 17665
Data columns (total 15 columns):
Movie_ID     2321 non-null int64
Title        2321 non-null object
Year         2321 non-null object
Rating       1886 non-null object
Companies    2313 non-null object
Country      2321 non-null object
Language     2321 non-null object
RunTime      2321 non-null float64
Crew         2257 non-null object
Cast         2190 non-null object
Genres       2320 non-null object
Budget       2321 non-null float64
Gross        2321 non-null float64
ScoreImdb    2321 non-null float64
VotesImdb    2321 non-null float64
dtypes: float64(5), int64(1), object(9)
memory usage: 290.1+ KB


In [8]:
# fig = SB.make_subplots(rows=1, cols=2, vertical_spacing=0.1, 
#                        subplot_titles=("Budget", "Gross"))

# fig.add_trace(
#     GO.Violin(y=scoreprofit_df['Budget'], meanline_visible=True, name=''), 
#     row=1, col=1)
# fig.add_trace(
#     GO.Violin(y=scoreprofit_df['Gross'], meanline_visible=True, name=''), 
#     row=1, col=2)

# fig.update_layout(
#     title={'text': "Financial EDA", 'x':0.5, 'y':0.97,  
#         'xanchor': 'center', 'yanchor': 'top'},
#     width=900,
#     height=400,
#     margin=GO.layout.Margin(t=50, r=20, b=20, l=70, pad=0),
#     paper_bgcolor="LightSteelBlue",
#     showlegend=False,
# )

In [9]:
# roi_df = roi_df.loc[ (roi_df['Budget']<201e6) ]
# roi_df.shape

In [10]:
# calculate new financial columns

scoreprofit_df['profit'] = (scoreprofit_df['Gross'] - scoreprofit_df['Budget']) 
scoreprofit_df.describe()

,Movie_ID,RunTime,Budget,Gross,ScoreImdb,VotesImdb,profit
count,2321.000000,2321.000000,2.321000e+03,2.321000e+03,2321.000000,2.321000e+03,2.321000e+03
mean,145283.246876,108.281775,1.856531e+07,4.647955e+07,6.247652,5.516566e+04,2.791424e+07
std,164791.100217,21.559898,3.133857e+07,1.177095e+08,1.145119,1.197701e+05,9.571809e+07
min,14.000000,32.000000,3.000000e+00,1.000000e+00,1.400000,1.600000e+01,-7.755132e+07
25%,13008.000000,93.000000,2.500000e+06,5.048580e+05,5.600000,3.217000e+03,-3.956329e+06
50%,49527.000000,103.000000,7.500000e+06,6.045657e+06,6.400000,1.276000e+04,3.294000e+04
75%,286521.000000,118.000000,2.000000e+07,3.715765e+07,7.100000,5.149500e+04,1.901293e+07
max,605373.000000,260.000000,3.210000e+08,2.048360e+09,8.900000,1.697791e+06,1.727360e+09


In [11]:
scoreprofit_df = scoreprofit_df.loc[ (scoreprofit_df['profit']<2e8) ]
scoreprofit_df.shape

(2232, 16)

In [26]:
# prepare columns for graphing

def ScoreProfitText(row):
    title = row['Title']
    profit = row['profit']
    score = row['ScoreImdb']
    hover_tx = f'{title}<br><b>{score}</b>  ${profit:.0e}'
    return hover_tx

scoreprofit_df['sp_text'] = scoreprofit_df.apply(ScoreProfitText, axis=1)
scoreprofit_df[['Title', 'profit', 'sp_text']][:5]

,Title,profit,sp_text
1,"10,000 BC",164784201.0,"10,000 BC<br><b>5.1</b> $2e+08"
26,Zombieland,78792080.0,Zombieland<br><b>7.6</b> $8e+07
33,Zone of the Dead,-968599.0,Zone of the Dead<br><b>3.9</b> $-1e+06
37,Zoology,-2980428.0,Zoology<br><b>6.4</b> $-3e+06
48,White Girl,-499758.0,White Girl<br><b>5.7</b> $-5e+05


In [27]:
# plot the score-profit curve

fig = GO.Figure()
fig.add_trace(GO.Scatter(x=scoreprofit_df['profit'], y=scoreprofit_df['ScoreImdb'],  
                         mode='markers', marker={'color': 'seagreen'},  
                         text=scoreprofit_df['sp_text'], hovertemplate ='<span>%{text}</span>', 
                         hoverlabel={'namelength': 0}))

fig.update_layout(
    title="IMDB Score vs Profit",
    xaxis_title="Profit ($ dollars)",
    yaxis_title="IMDB Score",
    width=600,
    height=500,
    margin=GO.layout.Margin(t=50, r=20, b=50, l=60, pad=0),
    paper_bgcolor="LightSteelBlue",
    #xaxis_type="log", 
)
# fig.update_xaxes(range=(-25e6, 225e6))
# fig.update_yaxes(range=(-100e6, 1e9))